<a href="https://colab.research.google.com/github/github.com/Gan4x4/ml_snippets/blob/main/Training/Lightning.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Torch pipeline
При работе с Pytorch базовый pipeline обучения выглядит примерно так

## Подготовка данных

In [ ]:
import torch
from torchvision import datasets, transforms, utils

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.13), (0.3))]
)

mnist = datasets.MNIST(root="./", train=True, download=True, transform=transform)

# Reduce size of dataset to speedup training
train_set, val_set, _ = torch.utils.data.random_split(mnist, [10000, 3000, 47000])

val_loader = torch.utils.data.DataLoader(val_set, batch_size=256, shuffle=False, num_workers=2)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True , num_workers=2)

## Создание модели

In [ ]:
from torch import nn

class SimpleModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.core = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28,256),
            nn.ReLU(),
            nn.Linear(256,10)
        )

  def forward(self, x):
    return self.core(x)

In [ ]:
model = SimpleModel()

## Код для валидации

In [ ]:
import torch

@torch.inference_mode()  # this annotation disable grad computation
def validate(model, test_loader,device):
    correct, total = 0, 0
    for imgs, labels in test_loader:
        pred = model(imgs.to(device))
        total += labels.size(0)
        _, predicted = torch.max(pred.data, 1) #shape = batch_size, class_count
        correct_predictions =  (predicted.cpu() == labels.cpu()).sum()
        correct += correct_predictions.sum().item()
    return correct / total


## Обучение (train loop)

In [ ]:
import torch
from tqdm import tqdm
# managing device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleModel()
model.to(device)

# define optimizer and loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # Weight update
criterion = nn.CrossEntropyLoss()  # Loss function
epochs = 3

for epoch in range(epochs):
  for batch in train_loader:
    # Processing one batch
    imgs, labels = batch
    optimizer.zero_grad()
    out = model(imgs)
    loss = criterion(out, labels.to(device))
    loss.backward()

    # Calclulate metrics: TODO
    # Save metrics to logs:  TODO

    optimizer.step()

  # Validation step
  print(f"Epoch {epoch} accuracy: {validate(model,val_loader,device):.2f}")
  # Save checkpoint: TODO

## Test

In [ ]:
test_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.13), (0.3))]
)
testset = datasets.MNIST(root="./", train=False, download=True, transform = test_transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=True)
accuracy = validate(model, test_loader, device)

print(f"Accuracy on TEST {accuracy:.2f}")

# Lightning

При обучении моделей в pytorch нам часто приходиться переписовать цикл обучения (train loop) это дублирование кода, которое нарушает принцип [DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself).

Кроме того нам нужно следить за процессом обучения модели, например если loss взрываться или выходит на плато как правило есть смысл остановить обучение. Чтобы контролировать этот процесс приходиться добавлять дополнительный код для вывода и/или логгирования метрик.

При проведении реальных экспериментов логирование результатов станет необходимым. Фреймворк ([Lightning](https://lightning.ai/)) облегчает написание tain loop, логирование результатов, и выполняет за нас ряд других задач.

In [ ]:
!pip install lightning

## Train loop в Lightning
Базовая задача которую решает Lightning это реализация train loop.

Типичный цикл обучения разбит на фрагменты каждый из которых помещен в соответствующий метод класса LightningModule.

In [ ]:
import lightning as L

class LitDemo(L.LightningModule):
    def __init__(self):
        super().__init__()

    def configure_optimizers(self):
        print("configure_optimizers")
        #return optimizer

    def on_train_epoch_start(self):
        print("on_train_epoch_start")

    def training_step(self, batch, batch_idx):
        #print("training_step")
        pass
        #return loss

    def on_validation_epoch_start(self):
        # called only if validation_step implemented
        print("on_validation_epoch_start")

    def validation_step(self, batch, batch_idx):
        #print("validation_step")
        pass

    def on_validation_epoch_end(self):
        print("on_validation_epoch_end")

    def on_train_epoch_end(self):
        print("on_train_epoch_end")

Что бы воспользоваться таким модулем надо передать его в объект класса Trainer

In [ ]:
#L.seed_everything(42)
lit_model = LiDemo()
trainer = L.Trainer(max_epochs=1)
trainer.fit(model=lit_model, train_dataloaders=train_loader, val_dataloaders= val_loader)

Видно, один цикл валидации запускается до до начала эпохи обучения, а затем повторяеттся внутри каждой эпохи.

Отключить первый вызов валидации можно инициализировав Trainer c параметром num_sanity_val_steps=0

In [ ]:
lit_model = LitDemo()
trainer = L.Trainer(max_epochs=1,
                    num_sanity_val_steps=0 # disable vlidation before first epoch
                    )
trainer.fit(model=lit_model, train_dataloaders=train_loader, val_dataloaders= val_loader)

### Перепишем цикл обучения из на Lightning.


Модель мы можем не менять, достаточно сохранить на не ссылку при инициализации.
Нужно инициализировать оптимизатор и перенести чать кода в train_step

In [ ]:
class LitMinimal(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.criterion = nn.CrossEntropyLoss()

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01,momentum =0.9)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = self.criterion(out, y)
        return loss

При создании оптимизатора мы передаем ему не параметры модели, а параметры всего модуля, поэтом не важно как будет называться свойство содержащее ссылку на модель.



In [ ]:
lit_model = LitMinimal(model)
trainer = L.Trainer(max_epochs=1)
trainer.fit(model=lit_model, train_dataloaders=train_loader, val_dataloaders= val_loader)

Код выше минимально необходимый для обучения, он незначительно упрощает создание train_loop

При этом фреймворк самостоятельно:
- обновляет веса модели
- сохраняет checkpoints на диск

 в нем нет методов для оценки точности или вывода графика loss.

### Вычисление метрик

Для выисления метрик установим пакет [torchmetrics](https://torchmetrics.readthedocs.io/en/stable/)

In [ ]:
!pip install torchmetrics

Метрики это объекты

In [ ]:
import torchmetrics

accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)
# Basic usage
preds = torch.tensor([1.,2.,3.])
labels = torch.tensor([1,2,9])
print("Accuracy",accuracy_metric(preds,labels))


Они могут накапливать данные а потом вычислять занчение метрики.

In [ ]:
print("Accuracy",accuracy_metric.compute()) # old values stored in memory

Если они не нужны следует их очистить

In [ ]:
accuracy_metric.reset() # lear old values
print("Accuracy",accuracy_metric.compute())

In [ ]:
for i in range(10):
  preds = torch.randint(0,10,(256,10)).float() # batch predictions
  labels = torch.randint(0,10,(256,)) # batch labels
  accuracy_metric.update(preds,labels)

print("Accuracy",accuracy_metric.compute())
accuracy_metric.reset()

# Логиррование в Lightning

Добавим подсчет метрики в lightning модуль. Будем добавлять значения в метрику при обработке каждого batch.

Выводить значения метрики будем в конце каждой эпохи обучения.

Для сохранения значений (метрик и любых других) в lightning модуле реализован метод `log`. Используем его так же для согранения loss на каждом batch.

Что бы последнее значение отображалось в progress bar установим параметр `prog_bar = True`

In [ ]:
class LitWithMetric(LitMinimal):
  def __init__(self, model):
      super().__init__(model)
      self.metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)

  def training_step(self, batch, batch_idx):
      x, y = batch
      out = self.model(x)
      loss = self.criterion(out, y)
      self.metric.update(out, y)
      self.log("loss", loss,prog_bar = True)
      return loss

  def on_train_epoch_end(self):
      self.log("accuracy/train", self.metric.compute(),prog_bar = True)
      self.metric.reset()


In [ ]:
model = SimpleModel()
lit_model = LitWithMetric(model)
trainer = L.Trainer(max_epochs=3) # def on_validation_epoch_start(self):
trainer.fit(model=lit_model, train_dataloaders=train_loader, val_dataloaders= val_loader)

In [ ]:
import lightning as L
import torchmetrics




class LitBasic(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.m = model
        self.criterion = nn.CrossEntropyLoss()
        self.metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)

    def on_train_epoch_start(self):
        self.metric.reset()
        print("on_train_epoch_start")

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        out = self.m(x)
        loss = self.criterion(out, y)
        self.metric.update(out, y)
        self.log("loss", loss,prog_bar = True)
        return loss

    def on_validation_epoch_start(self):
        print("on_validation_epoch_start")
        self.log("accuracy/train", self.metric.compute(),prog_bar = True)
        self.metric.reset()

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        out = self.m(x)
        self.metric.update(out,y)

    def on_validation_epoch_end(self):
        print("on_validation_epoch_end")
        self.log("accuracy/val", self.metric.compute(),prog_bar = True)
        self.metric.reset()

    def on_train_epoch_end(self):
        print("on_train_epoch_end")

    """          optimizers """
    #def configure_optimizers(self):
    #    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    #    return optimizer

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
        return optimizer


           log_every_n_steps = 1,

In [ ]:
https://pytorch-lightning.readthedocs.io/en/1.7.2/common/lightning_module.html#hooks

In [ ]:
import lightning.pytorch as pl
from torch import optim
import torchmetrics

class LitCNN(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.metric = torchmetrics.classification.MulticlassAccuracy(10)
        self.criterion = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = self.criterion(out, y)
        self.log("train_loss", loss)
        self.metric.update(out,y)
        return loss

    def on_train_epoch_end(self):
        self.log("accuracy/train", self.accuracy_train.compute())
        self.accuracy_train.reset()

    def on_validation_epoch_end(self):
        self.log("accuracy/val", self.accuracy_val.compute())
        self.accuracy_val.reset()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        self.accuracy_val.update(out, y)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.001)
        return optimizer

Shadow work


* managing devices
* creating checkpoints
* finding LR

Standartize

* train loop creation
* logging




# Experiment naming

https://lightning.ai/docs/pytorch/stable/extensions/generated/lightning.pytorch.loggers.TensorBoardLogger.html#lightning.pytorch.loggers.TensorBoardLogger

Logger setup
from lightning.pytorch.loggers import TensorBoardLogger

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = Trainer(logger=logger)

Log two variable in one axxis

In [ ]:
https://stackoverflow.com/questions/66287075/pytorch-lightning-multiple-scalars-e-g-train-and-valid-loss-in-same-tensorbo

# Checkpoint
- переименовывать ключи
https://lightning.ai/docs/pytorch/stable/common/checkpointing_basic.html

In [ ]:
https://lightning.ai/docs/pytorch/stable/common/checkpointing_basic.html

#Learning rate

https://lightning.ai/docs/pytorch/2.1.0/advanced/training_tricks.html#learning-rate-finder

# Test

https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.core.LightningModule.html#lightning.pytorch.core.LightningModule.test_step

https://pytorch-lightning.readthedocs.io/en/1.4.9/common/test_set.html

Дополнительно

### Abrcbhetv seed

In [ ]:
L.seed_everything(42)